In [3]:
import uproot, h5py
import awkward as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import os, glob
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplhep as hep
from skimage.measure import block_reduce
from numpy.lib.stride_tricks import as_strided
plt.style.use([hep.style.ROOT, hep.style.firamath])
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import LogNorm, ListedColormap, LinearSegmentedColormap
import matplotlib.patches as mpatches
# Define the CMS color scheme
cms_colors = [
    (0.00, '#FFFFFF'),  # White
    (0.33, '#005EB8'),  # Blue
    (0.66, '#FFDD00'),  # Yellow
    (1.00, '#FF0000')   # red
]

# Create the CMS colormap
cms_cmap = LinearSegmentedColormap.from_list('CMS', cms_colors)
from numpy.lib.stride_tricks import as_strided




In [2]:
def upsample_array(x, b0, b1):
    r, c = x.shape                                    # number of rows/columns
    rs, cs = x.strides                                # row/column strides
    x = as_strided(x, (r, b0, c, b1), (rs, 0, cs, 0)) # view as a larger 4D array
    return x.reshape(r*b0, c*b1)/(b0*b1) # create new 2D array with same total occupancy
    
def resample_EE(imgECAL, factor=2):
    # EE-
    imgEEm = imgECAL[:140-85] # EE- in the first 55 rows
    imgEEm = np.pad(imgEEm, ((1,0),(0,0)), 'constant', constant_values=0) # for even downsampling, zero pad 55 -> 56
    imgEEm_dn = block_reduce(imgEEm, block_size=(factor, factor), func=np.sum) # downsample by summing over [factor, factor] window
    imgEEm_dn_up = upsample_array(imgEEm_dn, factor, factor)/(factor*factor) # upsample will use same values so need to correct scale by factor**2
    imgECAL[:140-85] = imgEEm_dn_up[1:] ## replace the old EE- rows
    # EE+
    imgEEp = imgECAL[140+85:] # EE+ in the last 55 rows
    imgEEp = np.pad(imgEEp, ((0,1),(0,0)), 'constant', constant_values=0) # for even downsampling, zero pad 55 -> 56
    imgEEp_dn = block_reduce(imgEEp, block_size=(factor, factor), func=np.sum) # downsample by summing over [factor, factor] window
    imgEEp_dn_up = upsample_array(imgEEp_dn, factor, factor)# upsample will use same values so need to correct scale by factor*factor
    imgECAL[140+85:] = imgEEp_dn_up[:-1] # replace the old EE+ rows
    return imgECAL



def crop_jet(imgECAL, iphi, ieta, jet_shape=125):

    # NOTE: jet_shape here should correspond to the one used in RHAnalyzer
    off = jet_shape//2
    iphi = int(iphi*5 + 2) # 5 EB xtals per HB tower
    ieta = int(ieta*5 + 2) # 5 EB xtals per HB tower

    # Wrap-around on left side
    if iphi < off:
        diff = off-iphi
        img_crop = np.concatenate((imgECAL[:,ieta-off:ieta+off+1,-diff:],
                                   imgECAL[:,ieta-off:ieta+off+1,:iphi+off+1]), axis=-1)
    # Wrap-around on right side
    elif 360-iphi < off:
        diff = off - (360-iphi)
        img_crop = np.concatenate((imgECAL[:,ieta-off:ieta+off+1,iphi-off:],
                                   imgECAL[:,ieta-off:ieta+off+1,:diff+1]), axis=-1)
    # Nominal case
    else:
        img_crop = imgECAL[:,ieta-off:ieta+off+1,iphi-off:iphi+off+1]

    return img_crop

### Layesrs in images

In [4]:
layers =['TracksAtECAL_pt', 'TracksAtECAL_dZSig', 'TracksAtECAL_d0Sig', 'ECAL_energy', \
         'HBHE_energy', 'PixAtEcal_1','PixAtEcal_2', 'PixAtEcal_3', 'PixAtEcal_4', 'TibAtEcal_1',\
         'TibAtEcal_2','TobAtEcal_1','TobAtEcal_2', 'TibAtEcal_3','TibAtEcal_4', 'TobAtEcal_3', \
         'TobAtEcal_4', 'TobAtEcal_5', 'TobAtEcal_6' , 'TidAtEcal_1',\
                                  'TecAtEcal_1', 'TecAtEcal_2', 'TecAtEcal_3']
print("Number of layers", len(layers))


Number of layers 23


In [61]:

def convert_h5py_file(file, outdir):
    file=file
    outdir = outdir
    if not os.path.exists(outdir):
        os.makedirs(outdir)
        
    RHFile = uproot.open(file)
    RHTree = RHFile["fevt/RHTree"]
    branches = RHTree.arrays()
    outfile_name = f"{file.split('/')[-1].split('.')[0]}.h5"
    chunk_size = 2
    index_ = 0
    
    with h5py.File(f'{outdir}/{outfile_name}', 'w') as proper_data:
            dataset_names = ['all_jet', 'am', 'ieta', 'iphi', 'm0', 'a_pt', 'jet_pt']
            # dataset_names = ['all_jet', 'am', 'ieta', 'iphi']
            datasets = {
                name: proper_data.create_dataset(
                    name,
                    shape= (0,13, 125, 125) if 'jet' in name else (0,1),
                    maxshape=(None, 13, 125, 125) if 'jet' in name else (None, 1),
                    dtype='float32',  # Specify an appropriate data type
                    compression='lzf',
                    chunks=(chunk_size, 13, 125, 125) if 'jet' in name else (chunk_size, 1),
                ) for name in dataset_names
            }
    
            for i in range(len(branches["ECAL_energy"])):
                ECAL_energy = np.array(branches["ECAL_energy"][i]).reshape(280,360)
                # ECAL_energy = resample_EE(ECAL_energy)
                HBHE_energy_ = np.array(branches["HBHE_energy"][i]).reshape(56,72)
                HBHE_energy = upsample_array(HBHE_energy_, 5, 5) # (280, 360)
                TracksAtECAL_pt = np.array(branches["ECAL_tracksPt_atECALfixIP"][i]).reshape(280,360)
                TracksAtECAL_dZSig = np.array(branches["ECAL_tracksDzSig_atECALfixIP"][i]).reshape(280,360)
                TracksAtECAL_d0Sig = np.array(branches["ECAL_tracksD0Sig_atECALfixIP"][i]).reshape(280,360)
                PixAtEcal_1        = np.array(branches["BPIX_layer1_ECAL_atPV"][i]).reshape(280,360)
                PixAtEcal_2        = np.array(branches["BPIX_layer2_ECAL_atPV"][i]).reshape(280,360)
                PixAtEcal_3        = np.array(branches["BPIX_layer3_ECAL_atPV"][i]).reshape(280,360)
                PixAtEcal_4        = np.array(branches["BPIX_layer4_ECAL_atPV"][i]).reshape(280,360)
                TibAtEcal_1        = np.array(branches["TIB_layer1_ECAL_atPV"][i]).reshape(280,360)
                TibAtEcal_2        = np.array(branches["TIB_layer2_ECAL_atPV"][i]).reshape(280,360)
                TibAtEcal_3        = np.array(branches["TIB_layer3_ECAL_atPV"][i]).reshape(280,360)
                TibAtEcal_4        = np.array(branches["TIB_layer4_ECAL_atPV"][i]).reshape(280,360)
                TobAtEcal_1        = np.array(branches["TOB_layer1_ECAL_atPV"][i]).reshape(280,360)
                TobAtEcal_2        = np.array(branches["TOB_layer2_ECAL_atPV"][i]).reshape(280,360)
                TobAtEcal_3        = np.array(branches["TOB_layer3_ECAL_atPV"][i]).reshape(280,360)
                TobAtEcal_4        = np.array(branches["TOB_layer4_ECAL_atPV"][i]).reshape(280,360)
                TobAtEcal_5        = np.array(branches["TOB_layer5_ECAL_atPV"][i]).reshape(280,360)
                TobAtEcal_6        = np.array(branches["TOB_layer6_ECAL_atPV"][i]).reshape(280,360)
                TecAtEcal_1        = np.array(branches["TEC_layer1_ECAL_atPV"][i]).reshape(280,360)
                TecAtEcal_2        = np.array(branches["TEC_layer2_ECAL_atPV"][i]).reshape(280,360)
                TecAtEcal_3        = np.array(branches["TEC_layer3_ECAL_atPV"][i]).reshape(280,360)
                TidAtEcal_1        = np.array(branches["TID_layer1_ECAL_atPV"][i]).reshape(280,360)
                X_CMS             = np.stack([TracksAtECAL_pt, TracksAtECAL_dZSig, TracksAtECAL_d0Sig, ECAL_energy, HBHE_energy, PixAtEcal_1,\
                PixAtEcal_2, PixAtEcal_3, PixAtEcal_4, TibAtEcal_1, TibAtEcal_2, TobAtEcal_1,\
                                              TobAtEcal_2], axis=0) # (13, 280, 360)
                # X_CMS             = np.stack([TracksAtECAL_pt, TracksAtECAL_dZSig, TracksAtECAL_d0Sig, ECAL_energy, HBHE_energy, PixAtEcal_1,\
                # PixAtEcal_2, PixAtEcal_3, PixAtEcal_4, TibAtEcal_1, TibAtEcal_2, TobAtEcal_1,\
                #                               TobAtEcal_2,  TibAtEcal_3, TibAtEcal_4, TobAtEcal_3, TobAtEcal_4, TobAtEcal_5, TobAtEcal_6, TidAtEcal_1,\
                #                               TecAtEcal_1, TecAtEcal_2, TecAtEcal_3], axis=0) # (23, 280, 360)
                # ys = ak.Array(branches["jetIsSignal"])[i]
                ys = ak.Array(branches["jetIsDiTau"])[i]
                iphis  = ak.Array(branches["jetSeed_iphi"])[i]
                ietas  = ak.Array(branches["jetSeed_ieta"])[i]
                m0s    = ak.Array(branches["jetM"])[i]
                jetpts = ak.Array(branches["jetPt"])[i]
                ams = ak.Array(branches["a_m"])[i]
                apts = ak.Array(branches["a_pt"])[i]

                index_ = index_ + len(ys)
                print(f"working on jet {index_}")
                for name, dataset in datasets.items():
                    dataset.resize((index_,13, 125, 125) if 'jet' in name else (index_,1))

                for j in range(len(ys)):
                    proper_data['all_jet'][index_ - len(ys) + j, :, :, :] = crop_jet(X_CMS, iphis[j], ietas[j], jet_shape=125)
                    proper_data['am'][index_ - len(ys) + j, :] = ams[j]
                    proper_data['ieta'][index_ - len(ys) + j, :] = ietas[j]
                    proper_data['iphi'][index_ - len(ys) + j, :] = iphis[j]
                    proper_data['m0'][index_ - len(ys) + j, :] = m0s[j]
                    proper_data['a_pt'][index_ - len(ys) + j, :] = apts[j]
                    proper_data['jet_pt'][index_ - len(ys) + j, :] = jetpts[j]



In [62]:
convert_h5py_file(file=file, outdir='test')

working on jet 1
working on jet 2
working on jet 3
working on jet 4
working on jet 6
working on jet 7
working on jet 9
working on jet 11
working on jet 13


In [73]:
file = 'test/Ato2Tau_massreg_sample.h5'
data = h5py.File(f'{file}', 'r')



images_batch = data["all_jet"][0]
am_batch = data["am"][0]
# ieta_batch = data["ieta"][start_idx:end_idx, :]
# iphi_batch = data["iphi"][start_idx:end_idx, :]
# m0_batch = data["m0"][start_idx:end_idx, :]

print(images_batch.shape)    
print(len(data["am"]))
data.close()
    


(13, 125, 125)
13
